# 10-minute quick start

Installation:

In [1]:
%load_ext autoreload
%autoreload 2

from finestflow.base import Composable

In [2]:
class Plus(Composable):
    x: int
    y: int

    def run(self, z):
        return (self.x + self.y) * z

class Step2(Composable):
    a: int
    b: int

    def run(self):
        return self.a * self.b


def step3(a, b):
    return a + b


class Step4:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def calculate(self, c):
        return self.a + self.b + c

In [3]:
class Workflow(Composable):
    param1: int
    param2: int
    step5: Composable

    def initialize_nodes(self):
        self.step1 = Plus(x=10, y=20)
        self.step2 = Step2(a=self.param1, b=self.param2)
        self.step3 = step3
        self.step4 = Step4(10, 10)

    def run(self, x):
        a = self.step1(x, _ff_name="step1")
        a += self.step2(_ff_name="step2")
        a += self.step3(a, x, _ff_name="step3")
        a += self.step4.calculate(x, _ff_name="step4")
        a += self.step5(a, x, _ff_name="step5")
        return a

In [4]:
workflow = Workflow(param1=2, param2=5, step5=step3)

In [5]:
output = workflow(10)
print(f"- {output=}")
print(f"- {type(output)=}")

- output=1330
- type(output)=<class 'int'>


In [8]:
workflow.context._msg_store

{'__global_key__': {'run_id': '1690397696'},
 '': {},
 '__progress__': {'.step1': {'input': {'args': (10,),
    'kwargs': {'_ff_name': 'step1'}},
   'output': 300,
   'status': 'run'},
  '.step2': {'input': {'args': (), 'kwargs': {'_ff_name': 'step2'}},
   'output': 10,
   'status': 'run'},
  '.step3': {'input': {'args': (310, 10), 'kwargs': {'_ff_name': 'step3'}},
   'output': 320,
   'status': 'run'},
  '.step4': {'input': {'args': (10,), 'kwargs': {'_ff_name': 'step4'}},
   'output': 30,
   'status': 'run'},
  '.step5': {'input': {'args': (660, 10), 'kwargs': {'_ff_name': 'step5'}},
   'output': 670,
   'status': 'run'},
  '': {'input': {'args': (10,), 'kwargs': {'_ff_name': ''}},
   'output': 1330,
   'status': 'run'}},
 '.step1': {},
 '.step2': {}}

In [7]:
workflow.params

{'param1': 2, 'param2': 5}

In [16]:
workflow.last_run.output(".step3")

320

In [3]:
from typing import Optional

class Plus(Step):
    x: int
    y: Optional[int] = None

    def run(self, z):
        print(self.x, self.y)
        return (self.x + self.y) * z


In [4]:
p = Plus(x=100, y=20)
print(repr(p))
p.run(2)

Plus(x=100, y=20)
100 20


240

In [9]:
p(2, _ff_name="hehe")

100 20


240

In [11]:
p.context._msg_store

{'__global_key__': {},
 '__progress__': {'hehe': {'input': {'args': (2,),
    'kwargs': {'_ff_name': 'hehe'}},
   'output': 240,
   'status': 'run'}}}

In [13]:
p.last_run.logs()

{'hehe': {'input': {'args': (2,), 'kwargs': {'_ff_name': 'hehe'}},
  'output': 240,
  'status': 'run'}}

In [24]:
from typing import Optional, Callable

class Thought(Step):

    base_template: str
    base_format: Optional[dict] = None   # pyright: reportGeneralTypeIssues=false
    parser: Optional[Callable] = None

    def __init__(self, **params):
        super().__init__(**params)
        if self.base_format is None:
            self.base_format: dict = {}

    def format(self, **kwargs):
        self.base_format.update(kwargs)
        return self.template

    @property
    def template(self):
        remaining = {
            each: f"{{{each}}}" for each in set(self.fieldnames) - set(self.base_format.keys())
        }
        return self.base_template.format(**self.base_format, **remaining)

    @property
    def fieldnames(self):
        from string import Formatter

        return [
            fname for _, fname, _, _ in Formatter().parse(self.base_template) if fname
        ]

    def run(self, **kwargs):
        # response: openai.openai_object.OpenAIObject = openai.ChatCompletion.create(
        #     engine="gpt35turbo",
        #     temperature=0,
        #     request_timeout=60,
        #     messages=[
        #         {
        #             "role": "user",
        #             "content": self.format(**kwargs),
        #         }
        #     ],
        # )
        # text: str = response["choices"][0]["message"]["content"]
        text = "hahaha"
        if self.parser is not None:
            text = self.parser(text)
        return text


In [26]:
thought = Thought(
    base_template="HEHEHEHE {abc} and {xyz}"
)

In [27]:
thought.format(abc="hihihihi")

'HEHEHEHE hihihihi and {xyz}'

In [28]:
thought.template

'HEHEHEHE hihihihi and {xyz}'

In [29]:
thought.base_template

'HEHEHEHE {abc} and {xyz}'

In [30]:
thought.base_format

{'abc': 'hihihihi'}

In [31]:
thought.params

{'base_template': 'HEHEHEHE {abc} and {xyz}'}